In [56]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import collections

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics

sys.path.append("../tfti")
import tfti

# Reset graph for conistency.
tf.reset_default_graph()

# Prepare model.
problem_name = "genomics_binding_deepsea_gm12878"
model_name = "tfti_transformer"
hparams_set = "tfti_transformer_debug"
hparams_overrides_str = ""

data_dir = os.path.expanduser("~/t2t/data")
output_dir = os.path.expanduser(f"~/t2t_train/{problem_name}/{model_name}-{hparams_set}-{hparams_overrides_str}")

# Prepare model.
hparams = trainer_lib.create_hparams(hparams_set, hparams_overrides_str, data_dir, problem_name)
problem = registry.problem(problem_name)
encoders = problem.get_feature_encoders(data_dir)

# Prepare the model and the graph when model runs on features.
model = registry.model(model_name)(hparams, tf.estimator.ModeKeys.EVAL)

# Prepare features for feeding into the model.
inputs_ph = tf.placeholder(dtype=tf.string, shape=[])
targets_ph = tf.placeholder(dtype=tf.int64, shape=[problem.num_binary_predictions])
features = {"inputs": inputs_ph, "targets": targets_ph}
features = problem.preprocess_example(features, tf.estimator.ModeKeys.EVAL, hparams)

# Introduce a dummy batch dimension.
for key in features.keys():
    features[key] = tf.expand_dims(features[key], 0)

logits, losses = model(features)
labels = features["targets"]

# Evaluation metrics we want to use.
set_auroc, _ = tfti.set_auroc(logits, labels, features)
set_auprc, _ = tfti.set_auprc(logits, labels, features)
average_auroc, _ = tfti.average_auroc(logits, labels, features)
average_auprc, _ = tfti.average_auprc(logits, labels, features)

saver = tf.train.Saver()
sess = tf.InteractiveSession()

# Initialize AUC running average stuff.
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init_op)

# Load weights from checkpoint.
ckpts = tf.train.get_checkpoint_state(output_dir)
ckpt = ckpts.model_checkpoint_path
saver.restore(sess, ckpt)

INFO:tensorflow:Unsetting shared_embedding_and_softmax_weights.


[2018-04-20 20:24:50,280] Unsetting shared_embedding_and_softmax_weights.


INFO:tensorflow:Setting T2TModel mode to 'eval'


[2018-04-20 20:24:50,284] Setting T2TModel mode to 'eval'


INFO:tensorflow:Setting hparams.dropout to 0.0


...

[2018-04-20 20:24:50,721] Building model body


INFO:tensorflow:Transforming body output with binary_class_label_modality_8.top


[2018-04-20 20:24:52,779] Transforming body output with binary_class_label_modality_8.top


INFO:tensorflow:Restoring parameters from /Users/alexyku/t2t_train/genomics_binding_deepsea_gm12878/tfti_transformer-tfti_transformer_debug-/model.ckpt-1001


[2018-04-20 20:24:58,778] Restoring parameters from /Users/alexyku/t2t_train/genomics_binding_deepsea_gm12878/tfti_transformer-tfti_transformer_debug-/model.ckpt-1001


In [65]:
dummy_inputs = "".join(np.random.choice(list("NACTG"), problem.input_sequence_length))
dummy_targets = np.random.choice(2, [problem.num_binary_predictions])

print(dummy_inputs)
print(dummy_targets)

# Run on each line.
average_auroc_numpy = sess.run(average_auroc, feed_dict={
    inputs_ph: dummy_inputs,
    targets_ph: dummy_targets
})

print(f"AUROC is: {average_auroc_numpy}")

NAGNAGCTTTTCTTNANTTNACNNAGGNAATCNNCAGTNGCAGAANACACGAGGTGACNGNGCCGGCNAGNCCGAATTNCNTTTCNCANGNNNCNANANGANCGANTCNGAGACGTTTCNAAACAGTANNTNNTCTAANCAGTNNCCCCTGTNCGCGNNCATCNCGNGCNAGGCTTNTTNCCGNTTGGCNTGGCTCTTNGNTNNAAGTTCCCTNNCATTGAANTNTGCCNATNACCGNNNTNACNNTGNTCNCGCTNAAGTTNCACTAAANANAACANTNTGGTTGGNTAATCCAACAGAAACCNCGACTCNCATNNCNCGCATTGNTTTNGGAAAGANGNACNNCTNTACGANGGGACCTCTAATTTGACGTCCCCNATNCACTCCCTCTNGNATGGATCGAANACNGCCATAANGANCTCGTAAGGNANGAGATGGCGAGAGAANNCTGTGCNNNGNTNTGATCNTTATCCGAAAGGGACTGTTCATNACNCNATAAAGTNTCNNGGGCGTACNGGTCNTNAGCCATTNGTAGTAGGNGTTGNCGCCTTCTGNNTNANACTAGGCGATNGCGTNCTNACGAGAGAGNTGTNNCNANCNGNTGNCAAACANNTNCCCANNGANTCTCTAGGCCTTCTNGCTCNNNGGTGACGCTTANTATAGCATANCNGTGNCACCCTCTTAAACAGANGNNAAGACGGCTACTANNGACCTTTGTTNNTAAGACAGCANTCNCTNGGGATATTACGTTCCACATCGACTGATNCCGACCAACAAAACGATTTNNACAGGGCNCAGCTCACNNNTGGNNCNGANNAGGNTNGGTGCTTGAATTGNGNCAACCANTGCAAAGCNGCNCCNAAGCCNANANCGGNATGNTCANTNANNGGNTTNGANTTCTCGNGNTGGNCNGAANGNACNNCGCNCCTCCTGGNTNTGACCTAGCAGTTGCCAAATAGNCATCTTTTNTTCNGNAAGCNTANTCNACCTATCGNTATTC